In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Saint Anthony,US,2020-08-24 01:39:06,45.02,-93.22,82.99,57,1,3.36
1,1,Ushuaia,AR,2020-08-24 01:34:10,-54.80,-68.30,32.00,93,90,9.17
2,2,Punta Arenas,CL,2020-08-24 01:35:40,-53.15,-70.92,33.80,86,20,24.16
3,3,Chokurdakh,RU,2020-08-24 01:39:06,70.63,147.92,38.14,76,100,10.47
4,4,Kyabe,TD,2020-08-24 01:39:07,9.45,18.94,71.31,96,100,3.51


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [6]:
#Max Temperature Heatmap
gmaps.configure(api_key=g_key)

locations = city_data_df[['Lat','Lng']]
maxTemp = city_data_df['Max Temp']
adj_maxTemp_weights = [max(temp, 0) for temp in maxTemp]

fig = gmaps.figure(center = (30.0,31.0), zoom_level =1.5)

heatmap_layer = gmaps.heatmap_layer(locations, weights = adj_maxTemp_weights, 
                                    dissipating = False, max_intensity = 300, point_radius=4)


# Add the heatmap layer.
fig.add_layer(heatmap_layer)
# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

heat_layer = gmaps.heatmap_layer(locations, weights=temps)

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

In [10]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Saint Anthony,US,2020-08-24 01:39:06,45.02,-93.22,82.99,57,1,3.36
6,6,Hithadhoo,MV,2020-08-24 01:38:20,-0.60,73.08,82.31,80,100,14.47
10,10,Bengkulu,ID,2020-08-24 01:39:08,-3.80,102.27,82.74,68,54,4.94
17,17,Ponta Do Sol,PT,2020-08-24 01:39:10,32.67,-17.10,77.00,57,20,18.34
20,20,Bethel,US,2020-08-24 01:38:46,41.37,-73.41,75.20,83,1,4.70
21,21,Kapit,MY,2020-08-24 01:39:11,2.02,112.93,84.00,73,47,2.42
22,22,Faanui,PF,2020-08-24 01:38:42,-16.48,-151.75,77.13,79,9,18.77
36,36,Hilo,US,2020-08-24 01:39:15,19.73,-155.09,84.20,70,40,6.93
37,37,Alyangula,AU,2020-08-24 01:39:16,-13.85,136.42,82.40,34,12,10.29
40,40,Shenjiamen,CN,2020-08-24 01:39:17,29.96,122.30,83.86,77,0,9.69


In [12]:
preferred_cities_df.count()

City_ID       193
City          193
Country       193
Date          193
Lat           193
Lng           193
Max Temp      193
Humidity      193
Cloudiness    193
Wind Speed    193
dtype: int64

In [13]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Saint Anthony,US,82.99,45.02,-93.22,
6,Hithadhoo,MV,82.31,-0.60,73.08,
10,Bengkulu,ID,82.74,-3.80,102.27,
17,Ponta Do Sol,PT,77.00,32.67,-17.10,
20,Bethel,US,75.20,41.37,-73.41,
21,Kapit,MY,84.00,2.02,112.93,
22,Faanui,PF,77.13,-16.48,-151.75,
36,Hilo,US,84.20,19.73,-155.09,
37,Alyangula,AU,82.40,-13.85,136.42,
40,Shenjiamen,CN,83.86,29.96,122.30,


In [15]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [19]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [20]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [23]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [24]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [25]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))